# NLP - TP Final

Augusto Rabbia

Profesores:
 - Alan Geary
 - Constantino Ferrucci
 - Dolores Sollberger
 - Juan Pablo Manson

# Cargar Datos e Inicializar Modelos

In [96]:
import pandas as pd
import zipfile
import os

# LLM para generación
#from huggingface_hub import InferenceClient
from google import genai
from google.colab import userdata

# Sentence Embedding
from transformers import AutoTokenizer, AutoModel
import torch

import requests

TESTING = True

Obtenemos los datos del archivo .zip

In [53]:
with zipfile.ZipFile('Datos TP1.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        if os.path.basename(root).startswith('.'): # Ignorar ipynb checkpoint
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
list_files('datos')

datos/
    queries_clasificador.csv
    informacion/
        video1.txt
        review_externa.txt
        manual.txt
        df_foros_bgg.csv
        review_bgg.txt
        video3.txt
        video4.txt
        video2.txt
    relaciones/
        creditos_relaciones.csv
    estadisticas/
        estadisticas.csv


Definimos nuestro LLM que utilizaremos a lo largo del trabajo. Se escogió el modelo `Gemini 2.5 Flash` de Google.

In [55]:
class LanguageModel():
    def __init__(self, init_function, generation_function):
        self._model = init_function()
        self._generation_function = generation_function

    def generar_respuesta(self, prompt):
        return self._generation_function(self._model, prompt)

# Para replicar los resultados, antes insertar la llave de la API para utilizar el modelo Gemini de Google
GOOGLE_API_KEY = ""

def init_gemini():
    return genai.Client(api_key=GOOGLE_API_KEY)

def generar_gemini(model, prompt):
    # En este caso, model es un tipo Client
    response = model.models.generate_content(model="gemini-2.5-flash-preview-05-20",contents=[prompt])
    return response.text.strip()

# Instancia el modelo con Gemini
llm_model = LanguageModel(init_gemini, generar_gemini)

# Crear Bases de Datos de Búsqueda

## BD de Texto

BD híbrida con búsqueda por similitud de embeddings y por palabras claves con reranking.

Instalar e importar librerías

In [57]:
!pip install chromadb rank_bm25 FlagEmbedding --quiet

In [58]:
import numpy as np
import pandas as pd

# Splitting de texto
from langchain.text_splitter import RecursiveCharacterTextSplitter

# BD Vectorial
import chromadb

# BD de búsqueda por palabras clave
from rank_bm25 import BM25Okapi
# Usamos word_tokenizer para tokenizar en BM25Searcher
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

# Reranking
from FlagEmbedding import FlagReranker

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Preprocesar e importar datos

Importar datos de texto

In [59]:
df_foros_bgg = pd.read_csv('datos/informacion/df_foros_bgg.csv')
print(df_foros_bgg.shape)
df_foros_bgg.head()

(36, 3)


,Category,Link to Post,Conversation
0,Reviews,https://boardgamegeek.com/thread/2660465/revie...,Title: Review: Still fun after ten plays? My p...
1,Reviews,https://boardgamegeek.com/thread/2201438/tiny-...,Title: Tiny review of Tiny Towns\nOriginal pos...
2,Reviews,https://boardgamegeek.com/thread/2258475/teens...,Title: Teensy Towns (a Space-Biff! review)\nOr...
3,Reviews,https://boardgamegeek.com/thread/2208048/clear...,"Title: Clear Eyes, Tiny Towns, Can't Lose: A R..."
4,Reviews,https://boardgamegeek.com/thread/2291815/simpl...,"Title: Simple, Thinky and Brutal\nOriginal pos..."


Preparación de datos en formato de texto.

In [60]:
# Leer el texto del foro.
# Se eliminan datos añadidos en la anterior etapa que no serán de importancia para el modelo
textos = df_foros_bgg['Conversation'].map(lambda x: x.replace('Title: ', '').replace('Original post: ', '')).tolist()

for roota, dirs, files in os.walk('datos/informacion'):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(roota, file), 'r') as f:
                textos.append(f.read())

# Splitting
chunk_size = 768
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[".", "?", "!","\n"]
    )

textos_split = []
for texto in textos:
    textos_split.append(text_splitter.split_text(texto))

textos_split = [item for sublist in textos_split for item in sublist]

print(f"""Cantidad de textos: {len(textos)}
Cantidad de fragmentos de texto: {len(textos_split)}
Ejemplo: {textos_split[0]}""")

Cantidad de textos: 43
Cantidad de fragmentos de texto: 275
Ejemplo: Review: Still fun after ten plays? My pros and cons
Even though long extensive reviews seem to be the most popular reviews on BGG, I myself just don't have the time or the patience to read those.I prefer quick overviews where I can get some insights into why this game may or may not be attractive to me


### BD Vectorial

Con búsqueda por similitud.

In [61]:
class BDVectorial():
    def __init__(self, tokenizer, embedder, sentences):
        self._model_name = model_name
        self._tokenizer = tokenizer
        self._embedder = embedder

        self._sentences = sentences
        self._embedded_sentences = self._embed_sentences(self._sentences)

        self._chroma_client = chromadb.PersistentClient(path="./BD_vectorial")
        self._collection = self._chroma_client.get_or_create_collection(name="sentence_embeddings")
        self._collection.add(
            documents=self._sentences,
            embeddings=self._embedded_sentences.tolist(),
            ids=[f"id_{i}" for i in range(len(self._embedded_sentences))]
        )

    def _embed_sentences(self, sentences):
        inputs = self._tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self._embedder(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1)  # Mean pooling
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)  # L2 normalize
        return embeddings.cpu().numpy()

    def buscar(self, texto, n_resultados=5):
        texto_embed = self._embed_sentences([texto])
        results = self._collection.query(
            query_embeddings=texto_embed.tolist(),
            n_results=n_resultados
        )
        return results['documents'][0]

model_name = "intfloat/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedder = AutoModel.from_pretrained(model_name)

bd_vectorial = BDVectorial(tokenizer, embedder, textos_split)
if TESTING:
    query_prueba = "Hay alguna forma de engañar a mi oponente?"
    print(f"""Query: {query_prueba}
    Primera respuesta: {bd_vectorial.buscar(query_prueba)[:2]}""")
    bd_vectorial.buscar(query_prueba)[0]

Query: Hay alguna forma de engañar a mi oponente?
Primera respuesta: ['quieres un juego donde todos los jugadores juegan a la vez con muchas decisiones y un montón de edificios monísimos hoy te traemos Tiny towns en 90 segundos en Tiny towns Tendremos que construir nuestras pequeñas ciudades colocando primero los recursos y después utilizándolos para levantar edificios en cada turno un jugador se convertirá en el maestro constructor podrá elegir Qué recursos recibirán todos los jugadores y cada uno decidiremos en cuál de las 16 casillas de nuestro tablero lo queremos colocar al hacerlo el objetivo será formar patrones que coincidan con los necesarios para construir un edificio concreto cuando un patrón esté completo podremos retirar los recursos que Lo componen Y colocar el edificio en una de las casillas donde estos se encontraban cada edificio nos dará unas habilidades o formas de puntuar diferentes por ejemplo algunos puntos harán dependiendo de las cosas que tengan cerca otros de a

'quieres un juego donde todos los jugadores juegan a la vez con muchas decisiones y un montón de edificios monísimos hoy te traemos Tiny towns en 90 segundos en Tiny towns Tendremos que construir nuestras pequeñas ciudades colocando primero los recursos y después utilizándolos para levantar edificios en cada turno un jugador se convertirá en el maestro constructor podrá elegir Qué recursos recibirán todos los jugadores y cada uno decidiremos en cuál de las 16 casillas de nuestro tablero lo queremos colocar al hacerlo el objetivo será formar patrones que coincidan con los necesarios para construir un edificio concreto cuando un patrón esté completo podremos retirar los recursos que Lo componen Y colocar el edificio en una de las casillas donde estos se encontraban cada edificio nos dará unas habilidades o formas de puntuar diferentes por ejemplo algunos puntos harán dependiendo de las cosas que tengan cerca otros de acuerdo a lo que hay en su fila columna otros servirán para intercambia

### BD de Búsqueda por Palabras Clave

In [62]:
class BM25Searcher():
    def __init__(self, texto: list):
        self._texto = texto
        self._texto_token = [word_tokenize(t.lower()) for t in texto]
        self._bm25 = BM25Okapi(self._texto_token)

    def buscar(self, query, n_resultados=5):
        tokenized_query = word_tokenize(query.lower())
        scores = self._bm25.get_scores(tokenized_query)
        ranked_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:n_resultados]
        return [self._texto[i] for i in ranked_indices]

bm25_searcher = BM25Searcher(textos_split)

if TESTING:
    query_prueba = "Hay alguna forma de engañar a mi oponente?"
    print(f"""Query: {query_prueba}
    Primera respuesta: {bm25_searcher.buscar(query_prueba)[:2]}""")

Query: Hay alguna forma de engañar a mi oponente?
Primera respuesta: ['. Tras esto, se revela una carta de cada tipo (7 cartas en total) y se devuelve el resto a la caja.\nSe forma una reserva general con los recursos y los edificios.\nSe mezcla el mazo de cartas de edificio y se entregan 2 a cada jugador. De estas, cada jugador se queda con una y descarta la otra. El resto del mazo y las cartas descartadas se devuelven a la caja.\nCada jugador recibe un tablero personal y una pieza de monumento.\nSe escoge aleatoriamente al jugador inicial y se le entrega el marcador de maestro constructor', '¡Ah, los juegos de mesa! Solían ser los mismos de siempre, ¿verdad? Monopoly, Clue – divertidos por un tiempo, pero luego… meh. Luego llegó Carcassonne. Ese juego fue la droga de entrada, el que me mostró que los juegos de mesa podían ser más que solo tirar dados sin pensar. Pero no fue hasta que entré a Millennium Games (mi tienda local) que me di cuenta de lo PROFUNDO que era el agujero del con

### BD de Texto Híbrida

Con reranking.

In [63]:
class BDTexto_Hibrida():
    def __init__(self, bd_1: BDVectorial, bd_2: BM25Searcher):
        self._vector_searcher = bd_1
        self._bm25_searcher = bd_2
        self._reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

    def buscar(self, consulta, n_resultados=5):
        sem_docs = self._vector_searcher.buscar(consulta, n_resultados)
        bm_docs = self._bm25_searcher.buscar(consulta, n_resultados)

        # Combinar resultados por documento, por si hubieran repetidos
        combined_docs = list(set(sem_docs + bm_docs))

        # rerank
        scores = self._reranker.compute_score([[consulta, doc] for doc in combined_docs])

        reranked_docs = [doc for _, doc in sorted(zip(scores, combined_docs), reverse=True)]

        # Limitar a n_resultados
        return reranked_docs[:n_resultados]

bd_texto_hibrida = BDTexto_Hibrida(bd_vectorial, bm25_searcher)
if TESTING:
  query_prueba = "Hay alguna forma de engañar a mi oponente?"
  print(f"""Query: {query_prueba}
  Respuesta: {bd_vectorial_hibrida.buscar(query_prueba)[:2]}""")

Query: Hay alguna forma de engañar a mi oponente?
Respuesta: ['. Pero, como he dicho, podemos estar con mil ojos intentando prever la cadena de recursos que nos va a llegar para evitar callejones sin salida y sacar el máximo partido posible.\nUtilizaré como ejemplo al gran Race for the Galaxy (aquí su tochorreseña), probablemente el mejor juego de desarrollo de cartas que tiene como concepto diferencial la selección de acciones que da pie a este mismo juego psicológico. Yo quiero hacer varias acciones, pero solo puedo escoger una. Tengo que intentar prever la que van a escoger mis rivales para no «perder» mi turno y poder activar aquella que ellos no vayan a seleccionar. Es la misma idea, aunque, obviamente, con mucha menos parafernalia', '.\nLo normal será intentar construir lo antes posible dicho monumento para poder disfrutar de su beneficio. Al mantenerse oculta hasta el momento de revelarla, los jugadores no pueden prever qué será lo siguiente que pedirán, consiguiendo introducir 

## BD Tabular de Estadísticas

Con queries construidas por el modelo generativo de Gemini 2.5.

Leemos los datos y transponemos para simplificar la búsqueda, de forma tal que:

`df[df["Title"] == "Avg. Rating"]["Value"]`

Sea, en cambio,

`df["Avg. Rating"]`


In [64]:
df_estadisticas = pd.read_csv('datos/estadisticas/estadisticas.csv').set_index("Title").transpose()
df_estadisticas

Title,Avg. Rating,No. of Ratings,Difficulty according to the community,Comments,Fans,Overall Rank,Family Rank,All Time Plays,This Month,Own,...,Estimated time of play,Official recommended age,Community recommended age,Price on Amazon,Price on In Hat Inc,Price on Boardtopia,Price on Hobbies and Games,Price on Miniature Market,Price on Gamers Guild AZ,Price on Noble Knight Games
Value,7.211,"20,744",2.06 / 5,"3,037","1,018",462,121,"96,506",96,"34,184",...,45–60 Min Playing Time,14+,10+,$39.99,from $39.95,from $39.99,from C$56.99,from $23.99,from $39.99,from $30.00


Definimos la clase de la BD.

In [127]:
class BDTabular():
    def __init__(self, data: pd.DataFrame, lm: LanguageModel):
        self._data = data
        self._lm = lm
        self._prompt_base = self._crear_prompt_base(data.columns.tolist())

    def _crear_prompt_base(self, columnas):
        return (lambda cons: """
Eres un modelo encargado de convertir las consultas del usuario que vienen en lenguaje natural, convertirlas a código de pandas en python, la información del dataframe es la siguiente:
columnas: {col}
Y cada columna corresponde a un único valor, como si fuera un diccionario.

Tus instrucciones son:
1. Interpretar la consulta del usuario, teniendo en cuenta su intención.
2. No incluir ninguna información adicional, como instrucciones adicionales, comentarios o asignaciones extra (como `resultado = ...`). Solo el código que se puede ejecutar directamente para mostrar el resultado.
Debes hacer uso de la variable 'df' que ya está definida.

Ejemplo:
Consulta: Cuantos fans tiene, y cual es el precio en In Hat Inc?
Tu respuesta: df[["Fans","Price on In Hat Inc"]]

La consulta del usuario es la siguiente:
{cons}

RESPONDE UNICAMENTE CON CÓDIGO PYTHON CON PANDAS Y NADA MÁS, LA VARIABLE SE LLAMA "df".
""".format(col=columnas, cons=cons))

    def buscar(self, consulta):
        df = self._data
        prompt = self._prompt_base(consulta)
        respuesta = self._lm.generar_respuesta(prompt).replace('```python', '').replace('```', '').strip()

        if respuesta == "None":
            return None
        try:
            return eval(respuesta)
        except Exception as e:
            print(f"Error al evaluar el código: {e}")
            return None

bd_tabular = BDTabular(df_estadisticas, llm_model)

if TESTING:
    print(bd_tabular.buscar("Cual es el menor precio del juego?"))
    print(bd_tabular.buscar("Cuan dificil es el juego?"))
    print(bd_tabular.buscar("Cual es la duracion del juego?"))

Error al evaluar el código: "['Price on Amazon', 'Price on Boardtopia', 'Price on Miniature Market'] not in index"
None
Title Difficulty according to the community
Value                              2.06 / 5
Value        45–60  Min     Playing Time
Name: Estimated time of play, dtype: object


## BD de Grafos de Relaciones

Utilizando redisgraph.

In [72]:
!pip install redis redisgraph --quiet
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
!./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 7.0 MB/s eta 0:00:00
--2025-06-27 01:46:53--  http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
Resolving nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)... 185.125.190.83, 185.125.190.81, 91.189.91.82, ...
Connecting to nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)|185.125.190.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1318204 (1.3M) [application/vnd.debian.binary-package]
Saving to: ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’

libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  2.01MB/s    in 0.6s    

2025-06-27 01:46:54 (2.01 MB/s) - ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’ saved [1318204/1318204]

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2_amd6

In [73]:
import redis
from redisgraph import Graph, Node, Edge
import re

El conjunto de datos de relaciones contiene muchas relaciones que resultan en ruido y aumentan la complejidad sin proporcionar información realmente relevante al juego. Por lo tanto, realizamos una tarea de limpieza exaustiva para obtener una base de datos que maximise su utilidad.

In [74]:
df_relaciones = pd.read_csv("datos/relaciones/creditos_relaciones.csv")
df_relaciones = df_relaciones[((df_relaciones["Relación"] != "Related Game")
    | (df_relaciones["Objeto"] == "Tiny Towns"))
    & (df_relaciones["Relación"] != "Family")
    & (df_relaciones["Relación"] != "Categories")
    & (df_relaciones["Relación"] != "Mechanisms")
    & (df_relaciones["Relación"] != "Descripcion")
    & (df_relaciones["Relación"] != "Primary Name")].reset_index(drop=True)

# invertir relaciones "Related Game" donde tiny towns es objeto para que sea sujeto
# Para simplificar, creamos una máscara y reemplazamos
condición = (df_relaciones['Relación'] == 'Related Game') & (df_relaciones['Objeto'] == 'Tiny Towns')
df_relaciones.loc[condición, ['Sujeto', 'Objeto']] = df_relaciones.loc[condición, ['Objeto', 'Sujeto']].values
df_relaciones

,Sujeto,Relación,Objeto
0,Tiny Towns,Alternate Names,Městečka na dlani
1,Tiny Towns,Alternate Names,Miasteczka
2,Tiny Towns,Alternate Names,Les Petites Bourgades
3,Tiny Towns,Alternate Names,Крихітні Містечка
4,Tiny Towns,Alternate Names,Крошечные города
5,Tiny Towns,Alternate Names,มหานครย่อส่วน
6,Tiny Towns,Alternate Names,タイニータウン
7,Tiny Towns,Alternate Names,小小城鎮
8,Tiny Towns,Alternate Names,타이니 타운
9,Tiny Towns,Year Released,2019


Notamos que el grafo se vería como una estrella con todos los vértices saliendo de Tiny Towns.

In [125]:
class Graph_db:
    def __init__(self, lm:LanguageModel, df_relaciones:pd.DataFrame):
        self._llm = lm
        self._graph = self._crear_grafo(df_relaciones)
        self._prompt_base = self._crear_prompt_base([self._limpiar_texto_alfanum(rel) for rel in df_relaciones["Relación"].unique().tolist()])

    def _crear_prompt_base(self, rels):
        return (lambda query: """
Eres un asistente especializado en transformar consultas en lenguaje natural a relaciones semánticas representables en una base de grafos.

Dado el siguiente esquema conceptual:
- Hay un nodo central: 'Tiny Towns' (un juego), conectado con múltiples otros objetos a través de relaciones.
- Las relaciones posibles son:
  {rels}

Tus instrucciones son:
1. Interpretar la consulta del usuario, teniendo en cuenta su intención.
2. De esta consulta, obtener TODAS las relaciones que conectan a 'Tiny Towns' con otros objetos según la consulta, separadas por "|".
3. Si no hay ninguna relación aplicable, responde únicamente con "None".
4. No incluir ninguna información adicional, como instrucciones adicionales, comentarios o asignaciones extra (como `resultado = ...`). Solo el código que se puede ejecutar directamente para mostrar el resultado.


Ejemplo:
Consulta: Quién diseñó el juego, y quién fue el creador?
Tu respuesta: Designer|Graphic Designer|Developer

Consulta del usuario:
"{query}"

Responde a la consulta únicamente con las relaciones correspondientes (separadas por |) o 'None'.
""".format(rels=rels, query=query))

    def _limpiar_texto_alfanum(self, texto):
        # Elimina caracteres no alfanumericos, hace stripping y reemplaza espacios por '_'
        return re.sub(r'[^a-zA-Z0-9\s]', '', texto).strip().replace(' ', '_')

    def _crear_grafo(self, df):
        redis_client = redis.Redis(host='localhost', port=6379)
        graph_name = 'Relaciones Tiny Towns'
        graph = Graph(graph_name, redis_client)

        # Insertar nodos y relaciones
        created_nodes = {}
        for row in df_relaciones.itertuples():
            source = self._limpiar_texto_alfanum(row[1])
            target = self._limpiar_texto_alfanum(row[3])
            rel = self._limpiar_texto_alfanum(row[2])
            if source == "" or target == "" or rel == "":
                continue

            # Crear nodo origen si no existe
            if source not in created_nodes:
                created_nodes[source] = Node(label='Entity', properties={'name': source})
                graph.add_node(created_nodes[source])

            # Crear nodo destino si no existe
            if target not in created_nodes:
                created_nodes[target] = Node(label='Entity', properties={'name': target})
                graph.add_node(created_nodes[target])

            # Crear la relación
            edge = Edge(created_nodes[source], rel, created_nodes[target])
            graph.add_edge(edge)

        graph.commit()

        return graph

    def buscar(self, query:str)-> str:
        graph_query = self._llm.generar_respuesta(self._prompt_base(query))
        if graph_query == "None":
          return "Respuesta no encontrada en la base de datos."

        cypher_query = f"MATCH (s:Entity {{name: 'Tiny_Towns'}})-[r:{graph_query}]->(t:Entity) RETURN s.name, type(r), t.name"
        results = self._graph.query(cypher_query)

        if not results:
          return "Respuesta no encontrada en la base de datos."

        response = "Relaciones encontradas:\n"
        res_set = set()

        for record in results.result_set:
            res_set.add(tuple(record))

        for record in res_set:
            source, rel, target = record
            response += f"{source} --{rel}--> {target}\n"
        return response

db_relaciones = Graph_db(llm_model, df_relaciones)

if TESTING:
    print(db_relaciones.buscar("Qué otros nombres tiene el juego, y quien lo publico?"))

Relaciones encontradas:
Tiny_Towns --Alternate_Names--> Miasteczka
Tiny_Towns --Publishers--> Lucky_Duck_Games
Tiny_Towns --Publishers--> Arrakis_Games
Tiny_Towns --Publishers--> Lanlalen
Tiny_Towns --Publishers--> All_In_Games
Tiny_Towns --Publishers--> Raven_Distribution
Tiny_Towns --Publishers--> Fractal_Juegos
Tiny_Towns --Publishers--> Reflexshop
Tiny_Towns --Publishers--> Galpagos_Jogos
Tiny_Towns --Publishers--> GaGa_Games
Tiny_Towns --Publishers--> KenBill
Tiny_Towns --Publishers--> Lord_of_Boards
Tiny_Towns --Publishers--> Hid_Konem
Tiny_Towns --Publishers--> Alderac_Entertainment_Group
Tiny_Towns --Alternate_Names--> Msteka_na_dlani
Tiny_Towns --Publishers--> Broadway_Toys_LTD
Tiny_Towns --Publishers--> White_Goblin_Games
Tiny_Towns --Alternate_Names--> Les_Petites_Bourgades
Tiny_Towns --Publishers--> CMON_Global_Limited
Tiny_Towns --Publishers--> Korea_Boardgames
Tiny_Towns --Publishers--> REXhry
Tiny_Towns --Publishers--> Pegasus_Spiele



# Clasificador de consultas

Se compararán dos clasificadores y se evaluará su performance:
- Evaluador con Regresión Logística
- Evaluador LLM con entrenamiento few-shot

In [100]:
# Procesamiento de datos
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Archivo CSV de consultas generado con Claude Sonnet 4

In [101]:
df_queries = pd.read_csv("datos/queries_clasificador.csv")
X_train, X_test, y_train, y_test = train_test_split(df_queries["query"], df_queries["categoria"], test_size=0.2, random_state=42)

## Clasificador LR

Se hará embedding del texto utilizando el modelo `e5-multilingual-small`.

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
class Clasificador_LR():
    def __init__(self, X_train_string, y_train, tokenizer, embedder, random_state=None):
        self._model = LogisticRegression(class_weight='balanced', random_state=random_state)

        self._tokenizer = tokenizer
        self._embedder = embedder
        X_train_embed = self._embed_sentences(X_train_string)

        self._model.fit(X_train_embed, y_train)

    def _embed_sentences(self, sentences):
        inputs = self._tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self._embedder(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1)  # Mean pooling
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)  # L2 normalize
        return embeddings.cpu().numpy()

    def predecir(self, data):
        # Si es un único valor, convertir en lista
        if isinstance(data, str):
            print("is string")
            data = [data]
        text_embedded = self._embed_sentences(data)
        return self._model.predict(text_embedded)

In [104]:
clasificador_lr = Clasificador_LR(X_train.to_list(), y_train, tokenizer, embedder, random_state=42)

In [105]:
print(classification_report(y_test, clasificador_lr.predecir(X_test.to_list())))
clasificador_lr.predecir("¿Quién inventó el juego?")[0]

              precision    recall  f1-score   support

estadisticas       0.64      0.62      0.63        26
 informacion       0.92      0.90      0.91       101
  relaciones       0.65      0.79      0.71        14

    accuracy                           0.84       141
   macro avg       0.74      0.77      0.75       141
weighted avg       0.84      0.84      0.84       141

is string


'relaciones'

## Clasificador LLM

In [106]:
class Clasificador_LLM():
    def __init__(self, lm: LanguageModel, relaciones, estadisticas):
        self._lm = lm
        self._prompt_base = self._crear_prompt_base(relaciones, estadisticas)

    def _crear_prompt_base(self, rels,stats):
        return (lambda query: f"""
Eres un agente especializado con la tarea de clasificar consultas sobre un juego llamado Tiny Towns.
Debes clasificar las consultas en una de las siguientes 3 categorías, correspondientes a la base de datos donde se encuentra esta información:
1. relaciones: Preguntas sobre relaciones del juego con otras entidades. En particular, alguna de las siguientes relaciones: {rels}
2. informacion: Preguntas sobre información del juego, como estrategias, reglas, partidas, mecánicas, qué hacer en ciertas situaciones, etc.
3. estadisticas: Preguntas sobre datos numéricos o estadísticos del juego. En particular, alguno de los siguientes datos: {stats}.
4. None: Si la consulta no se puede clasificar en ninguna de las categorías anteriores

Tus instrucciones son:
1. Interpretar la consulta del usuario, teniendo en cuenta su intención.
2. De esta consulta, obtener la base de datos donde se encontraría esa información.
3. No incluir ninguna información adicional, como instrucciones adicionales, comentarios o asignaciones extra (como `resultado = ...`). Solo el código que se puede ejecutar directamente para mostrar el resultado.

Ejemplos:
Consulta: 'Quien es el creador del juego?'
Tu respuesta: relaciones
Pregunta: 'Qué juego relacionado existe?'
Respuesta: relaciones
Consulta: 'En qué año salió el juego?'
Tu respuesta: relaciones
Consulta: 'Quién diseñó el juego, y quién fue el creador?'
Tu respuesta: relaciones

Pregunta: 'Como se gana?'
Respuesta: informacion
Pregunta: 'Qué hacer si pierdo mi ultima ficha?'
Respuesta: informacion
Pregunta: 'Que es lo más divertido de Tiny Towns?'
Respuesta: informacion
Pregunta: 'Que es lo más divertido de Tiny Towns?'
Respuesta: informacion
Pregunta: 'Qué estrategia recomiendas para remontar si se tuvo un mal comienzo de partida?'
Respuesta: informacion

Pregunta: 'Cuantos jugadores admite el juego?'
Respuesta: estadisticas
Pregunta: 'Cuantos jugadores activos tiene el juego?'
Respuesta: estadisticas
Pregunta: 'Cuál es el precio en Amazon?'
Respuesta: estadisticas
Pregunta: 'En qué puesto se posiciona para jugar en familia?'
Respuesta: estadisticas

Pregunta: 'A qué edad murió San Martín?'
Respuesta: None
Pregunta: 'Qué día tan bonito!'
Respuesta: None
Pregunta: 'Qué opinas sobre la película de Toy Story, está buena?'
Respuesta: None

Tu consulta es la siguiente:
{query}

Clasifica la consulta y responde solo con una de las siguientes palabras exactas: 'relaciones', 'informacion' o 'estadisticas'.
""".format(query=query))

    def predecir(self, query):
        prompt = self._prompt_base(query)
        respuesta = self._lm.generar_respuesta(prompt).replace('```python', '').replace('```', '').strip().lower()
        # Asegura que la respuesta sea válida
        if respuesta not in {"relaciones", "informacion", "estadisticas"}:
            return "No se encontró la información en ninguna de las bases de datos."
        return respuesta

In [107]:
estadisticas = ["Price", "Recommended age", "Number of players", "Rank"]
for stat in df_estadisticas.columns:
    if not stat.startswith("Price") and "age" not in stat and "players" not in stat and "Rank" not in stat:
        estadisticas.append(stat)
estadisticas

['Price',
 'Recommended age',
 'Number of players',
 'Rank',
 'Avg. Rating',
 'No. of Ratings',
 'Difficulty according to the community',
 'Comments',
 'Fans',
 'All Time Plays',
 'This Month',
 'Own',
 'Prev. Owned',
 'For Trade',
 'Want In Trade',
 'Wishlist',
 'Has Parts',
 'Want Parts',
 'Number of expansions',
 'Estimated time of play']

In [108]:
relaciones = df_relaciones["Relación"].unique().tolist()
clasificador_llm = Clasificador_LLM(llm_model, relaciones, estadisticas)

if TESTING:
    print(clasificador_llm.predecir("¿Quién inventó el juego?"))
    print(clasificador_llm.predecir("¿Cuál fue la partida más intensa de Tiny Towns?"))
    print(clasificador_llm.predecir("¿Cuántas expansiones tiene el juego?"))
    print(clasificador_llm.predecir("¿Qué día cae navidad en 2025?"))

relaciones
informacion
estadisticas
No se encontró la información en ninguna de las bases de datos.


## Conclusión y elección del modelo

Se puede observar que aunque el clasificador por regresión logística tenga un costo mucho menor, este tiene un performance insuficiente, cometiendo errores en 1 de cada 6 consultas.

Por otro lado, el clasificador LLM respondió correctamente a las consultas hechas. No sólo eso, también es razonable suponer que el clasificador LLM tendrá mayores capacidades de generalización al estar pre-entrenado y ser de uso general, pudiendo razonar de forma zero-shot la categoría a asignar a consultas diferentes a cualquier otra en el dataset. Ahora bien, no resulta una opción fácil de probar, al consumir demasiado rápidamente la cuota de prompts que nos proporcionan los servicios de LLMs por APIs.

Finalmente, elegimos el clasificador LLM por sus capacidades superiores.

# Pipeline de Recuperación

Clase Recuperador que integra al clasificador y las bases de datos y maneja errores para simplificar la búsqueda.

In [128]:
class Recuperador():
    """
    Recuperador de información a partir de las bases de datos para RAG.
    """
    def __init__(self, bd_info: BDTexto_Hibrida, bd_stats: BDTabular, bd_rels: BDGraph, clasificador = None):
        self._bd_info = bd_info
        self._bd_stats = bd_stats
        self._bd_rels = bd_rels
        self._clasificador = clasificador

    def recuperar(self, query: str):
        if self._clasificador is None:
            raise Exception("El clasificador no ha sido inicializado.")
        clase = self._clasificador.predecir(query)
        if clase == "informacion":
            return self._bd_info.buscar(query, n_resultados=5)
        elif clase == "estadisticas":
            return self._bd_stats.buscar(query)
        elif clase == "relaciones":
            return self._bd_rels.buscar(query)
        else:
            return "No se encontró la información en ninguna de las bases de datos."

# Chatbot

Definimos la clase de Chatbot.

In [129]:
class Chatbot():
    def __init__(self, recuperador: Recuperador, llm: LanguageModel):
        self._recuperador = recuperador
        self._llm = llm

    def chat_loop(self):
        prompt = self._generar_prompt_inicial()
        print("¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?")
        while True:
            query = input("> ")
            if query.lower() in ["salir", "exit", "quit", "q"]:
                print("¡Hasta luego!")
                break

            contexto_rag = self._recuperador.recuperar(query)

            prompt = prompt.format(query, contexto_rag)

            respuesta = self._llm.generar_respuesta(prompt)

            print(respuesta)

            prompt = self._continuar_prompt_contexto(query, respuesta)

    def _generar_prompt_inicial(self):
        return """Eres un Chatbot con la tarea de asistir al usuario con preguntas sobre el juego de mesa Tiny Towns.
Tu tarea es responder a las preguntas del usuario sobre este juego, utilizando información que se te proporcionará sistemáticamente desde una base de datos.

Tus instrucciones son:
1. Interpretar la consulta del usuario, teniendo en cuenta su intención.
2. Utilizar la información que se te proporcionará en la sección "Contexto".
3. Si la información y el contexto no fueran suficientes para responder a la pregunta, debes pedir al usuario que reformule su pregunta.
4. Responder en idioma español siempre.
5. Si la pregunta no trata sobre Tiny Towns, dile al usuario que no eres capaz de responder a la pregunta, pues eres un Chatbot especializado en Tiny Towns.

<INICIA CONVERSACION>

Tú: ¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?

<PRIMERA CONSULTA> (Recuerda que si la información es insuficiente, o la pregunta no es sobre Tiny Towns, debes pedir al usuario que reformule su pregunta)

Consulta del usuario: {}

Contexto: {}
"""

    def _continuar_prompt_contexto(self, query_anterior, respuesta):
        return query_anterior + "\n\n Tú:" + respuesta + """

<SIGUIENTE CONSULTA> (Recuerda que si la información es insuficiente, o la pregunta no es sobre Tiny Towns, debes pedir al usuario que reformule su pregunta)

Consulta del usuario: {}

Contexto: {}

"""

In [131]:
recuperador = Recuperador(bd_info=bd_texto_hibrida, bd_stats=bd_tabular, bd_rels=db_relaciones, clasificador=clasificador_llm)
if TESTING:
    Chatbot(recuperador, llm_model).chat_loop()

¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?
> Quien invento el juego?
El diseñador de Tiny Towns es Peter McPherson. Josh Wood es el desarrollador del juego.
> Cuantos jugadores recomendarias para una partida entre colegas?
Para una partida entre colegas, la comunidad recomienda 3 jugadores como el número ideal. El juego se puede jugar de 1 a 5 personas.
> Qué estrategia recomiendas para un novato?


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Para un jugador novato en Tiny Towns, la estrategia más recomendada y un buen punto de partida es **enfocarse en construir tu monumento único lo antes posible**. Al comienzo de la partida, cada jugador recibe un par de cartas para construir un edificio con un efecto exclusivo para su ciudad y un patrón distinto.

Intentar construir este monumento rápidamente te permitirá disfrutar de su beneficio especial durante la mayor parte de la partida, lo cual puede darte una ventaja significativa y una guía clara para tus primeras decisiones. Es una forma efectiva de "apostar por una estrategia e intentar llevarla a cabo" desde el principio.
> En qué año salió el juego?
El juego Tiny Towns fue lanzado en el año **2019**.
> En qué día cae navidad de 2025?
No tengo información sobre la fecha de Navidad en 2025. Por favor, reformula tu pregunta si es sobre Tiny Towns.
> q
¡Hasta luego!


# Agente ReAct

Instalamos e importamos librerías

In [132]:
!pip install duckduckgo_search wikipedia --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.9 MB/s eta 0:00:00


In [151]:
import wikipedia
from duckduckgo_search import DDGS
import logging
from typing import Dict, List, Tuple

# --- Configuración del Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

Definimos un nuevo recuperador, esta vez sin un clasificador.

In [153]:
class RecuperadorSinClasificador(Recuperador):
    """Encapsula todas las funciones que el agente puede llamar."""
    def __init__(self, bd_info: BDTexto_Hibrida, bd_stats: BDTabular, bd_rels: BDGraph):
        super().__init__(bd_info, bd_stats, bd_rels, clasificador=None)
        self._ddgs = DDGS()
        wikipedia.set_lang("en")

    def recuperar_sin_clasificador(self, query: str, bd: str):
        if bd == "db_info":
            return self._bd_info.buscar(query, n_resultados=5)
        elif bd == "db_stats":
            return self._bd_stats.buscar(query)
        elif bd == "db_rels":
            return self._bd_rels.buscar(query)
        elif bd == "wikipedia":
            """Busca un resumen sobre un tema en Wikipedia en español."""
            logger.info(f"Ejecutando buscar_wikipedia con query: '{query}'")
            try:
                wikipedia.set_lang('es')
                # auto_suggest=False para evitar que Wikipedia cambie la query
                result = wikipedia.summary(query, sentences=3, auto_suggest=True)
                return result
            except wikipedia.exceptions.PageError:
                return f"Error: No se encontró la página para '{query}'. Intenta con otro término."
            except wikipedia.exceptions.DisambiguationError as e:
                return f"Error: La búsqueda '{query}' es ambigua. Opciones: {e.options[:3]}"
            except Exception as e:
                return f"Error inesperado en Wikipedia: {e}"
        elif bd == "internet":
            resultados_busqueda = self._ddgs.text(query, max_results=3)
            if resultados_busqueda:
                return [resultado["body"] for resultado in resultados_busqueda]
            return "No se encontraron resultados en Internet."
        else:
            return "Error al buscar información."

Definimos el prompt para el agente.

In [155]:
SYSTEM_PROMPT = """Eres un asistente que DEBE seguir el método ReAct paso a paso. NUNCA puedes responder sin usar herramientas.

HERRAMIENTAS DISPONIBLES:
1. buscar_db_info(query: str): Busca información sobre estrategias, reglas, partidas, mecánicas, qué hacer en ciertas situaciones, etc.
2. buscar_db_stats(query: str): Busca datos numéricos o estadísticos del juego. En particular: Price, Recommended age, Number of players, Rank, Avg. Rating, No. of Ratings, Difficulty, Comments, Fans, All Time Plays, This Month, Own, Prev. Owned, For Trade, Want In Trade, Wishlist, Has Parts, Want Parts, Number of expansions, Estimated time of play.
3. buscar_db_rels(query: str): Busca relaciones del juego con otras entidades. En particular: Alternate Names, Year Released, Designer, Related Game, Artists, Publishers, Developer, Graphic Designer
4. buscar_wikipedia(query: str): Busca información general en Wikipedia. Utiliza esta herramienta SOLAMENTE si la pregunta no puede ser respondida utilizando las bases de datos locales.
5. buscar_internet(query: str): Busca información en Internet. Utiliza esta herramienta como último recurso si las otras herramientas no proporcionan la información necesaria, o para buscar información muy específica o actualizada que no esperas encontrar en las bases de datos locales.

REGLAS ABSOLUTAS - NO NEGOCIABLES:
1. PROHIBIDO inventar información o responder sin herramientas
2. PROHIBIDO dar Final Answer sin haber usado herramientas para TODA la información requerida
3. OBLIGATORIO usar herramientas para cada parte de la pregunta
4. OBLIGATORIO esperar Observation antes de continuar

PROCESO OBLIGATORIO:
1. Identifica QUÉ información necesitas
2. USA herramientas para obtener CADA pieza de información
3. Solo da Final Answer cuando tengas TODAS las Observations necesarias
4. Cuando ya sepas cuál es la respuesta, primero dilo en Thoughts, y LUEGO escribe el Final Answer (Ejemplo más adelante)

FORMATO ESTRICTO:
Thought: Qué necesito hacer ahora
Action: herramienta("parametro")

EJEMPLO PASO A PASO:
Pregunta: "Quién es el creador de Tiny Towns, y de qué país es oriundo?"

Paso 1:
Thought: Necesito buscar información sobre el creador de Tiny Towns. No conozco detalles específicos sin usar herramientas.
Action: buscar_db_rels("Quien es el creador de Tiny Towns?")

Paso 2 (después de recibir Observation):
Thought: El creador de Tiny Towns es [nombre]. Ahora necesito obtener su país de origen.
Action: buscar_wikipedia("[nombre]")

Paso 3 (después de recibir segunda Observation):
Thought: Tengo toda la información necesaria para responder la pregunta de ambas herramientas. Puedo responder basándome en las Observaciones recibidas.
Final Answer: [Respuesta usando SOLO información de las Observations]

RESTRICCIONES CRÍTICAS:
- NUNCA respondas preguntas que no sean sobre Tiny Towns. Si el usuario hace una consulta que no sea sobre el juego, responde: "Lo siento, solo puedo responder preguntas relacionadas con el juego de mesa Tiny Towns".
- Si una herramienta no devuelve resultados útiles, puedes usar otra como respaldo (por ejemplo, buscar_wikipedia o buscar_internet).

TU PRIMER PASO DEBE SER SIEMPRE UN "Action:" - NO PUEDES EMPEZAR CON "Final Answer"."""

Definimos la clase del Agente.

In [171]:
class AgenteChatbot():
    def __init__(self, recuperador: RecuperadorSinClasificador, llm: LanguageModel):
        self._recuperador = recuperador
        self._llm = llm
        self._acciones_validas = ["buscar_db_info", "buscar_db_stats", "buscar_db_rels", "buscar_wikipedia", "buscar_internet"]

    def chat_loop(self):
        print("¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?")
        while True:
            query = input("> ")
            if query.lower() in ["salir", "exit", "quit",  "q"]:
                print("¡Hasta luego!")
                break

            respuesta = self._iniciar_conversacion(query)
            print(respuesta)

    def _ejecutar_accion(self, funcion: str, query_generada: str):
        try:
            if funcion == "buscar_db_info":
                return self._recuperador.recuperar_sin_clasificador(query_generada, "db_info")
            elif funcion == "buscar_db_stats":
                return self._recuperador.recuperar_sin_clasificador(query_generada, "db_stats")
            elif funcion == "buscar_db_rels":
                return self._recuperador.recuperar_sin_clasificador(query_generada, "db_rels")
            elif funcion == "buscar_wikipedia":
                return self._recuperador.recuperar_sin_clasificador(query_generada, "wikipedia")
            elif funcion == "buscar_internet":
                return self._recuperador.recuperar_sin_clasificador(query_generada, "internet")
            else:
                return "Ocurrió un error al ejecutar la acción."
        except Exception as e:
            logger.error(f"Error ejecutando la herramienta {funcion}: {e}")
            return f"Error al ejecutar la herramienta: {e}"

    def _validar_y_extraer_accion(self, linea_accion: str) -> Tuple[bool, str, str]:
        """Valida el formato 'herramienta("parametro")' y extrae sus componentes."""
        try:
            # Remover espacios y buscar el patrón herramienta(parametro)
            linea_limpia = linea_accion.strip()
            if '(' not in linea_limpia or ')' not in linea_limpia:
                return False, "", ""

            funcion = linea_limpia.split('(')[0].strip()
            if funcion not in self._acciones_validas:
                return False, "", ""

            # Extraer parámetro entre paréntesis
            inicio_param = linea_limpia.find('(') + 1
            fin_param = linea_limpia.rfind(')')
            parametro_bruto = linea_limpia[inicio_param:fin_param]
            parametro = parametro_bruto.strip().strip('"\'')

            return True, funcion, parametro
        except Exception as e:
            logger.error(f"Error validando acción: {e}")
            return False, "", ""

    def _parse_output_llm(self, output_modelo: str):

        paso_actual_str = []
        terminado = False

        thought = None
        action = None
        final_answer = None

        # Parsear la respuesta del LLM línea por línea
        lineas = output_modelo.strip().split('\n')

        # Verificar si está intentando dar respuesta final sin usar herramientas
        primera_linea_significativa = None
        for linea in lineas:
            linea_clean = linea.strip()
            if linea_clean:
                primera_linea_significativa = linea_clean.lower()
                break

        # Si empieza con final answer, rechazar y forzar uso de herramientas
        if primera_linea_significativa and (primera_linea_significativa.startswith("final answer:") or primera_linea_significativa.startswith("answer:")):
            return "Thought: No puedo dar una respuesta final sin usar herramientas. Debo buscar la información requerida paso a paso. Ahora debo comenzar con una sección 'Thought' antes de continuar.", False

        for i, linea in enumerate(lineas):
            linea = linea.strip()
            if not linea:
                continue

            linea_lower = linea.lower()

            if linea_lower.startswith("thought:"):
                thought = linea
            elif linea_lower.startswith("action:"):
                action = linea
            elif linea_lower.startswith("final answer:") or linea_lower.startswith("answer:"):
                final_answer = '\n'.join(lineas[i:])
                terminado = True
                break

        # Construir el paso de forma controlada
        if thought:
            paso_actual_str.append(thought)

        if action and not terminado:  # Solo ejecutar acción si no hay respuesta final
            paso_actual_str.append(action)
            accion_contenido = action[7:].strip()  # Remover "Action: "
            es_valida, nombre, param = self._validar_y_extraer_accion(accion_contenido)
            if es_valida:
                observacion = self._ejecutar_accion(nombre, param)
                paso_actual_str.append(f"Observation: {observacion}")
            else:
                paso_actual_str.append("Observation: Error en el formato de la acción. Usa: herramienta(\"parametro\")")

        if final_answer:
            paso_actual_str.append(final_answer)

        # Si no hay contenido válido, forzar el siguiente paso
        if not paso_actual_str:
            return "Thought: Debo usar herramientas para obtener la información solicitada. Empezaré con la primera parte de la pregunta.", False

        return "\n".join(paso_actual_str), terminado

    def _iniciar_conversacion(self, query: str, max_iteraciones: int = 5) -> str:
        """Gestiona el ciclo completo de la conversación ReAct."""

        system_prompt = SYSTEM_PROMPT

        user_prompt = f"Pregunta del usuario: {query}"

        mensajes = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]

        historial_completo = [user_prompt]
        iteracion = 0

        while iteracion < max_iteraciones:
            iteracion += 1
            logger.info(f"--- Iteración {iteracion} ---")

            try:
                query = "\n".join([f"{m['role']}: {m['content']}" for m in mensajes])
                respuesta_llm = self._llm.generar_respuesta(query)

                # Procesar la respuesta y ejecutar la acción
                paso_procesado, terminado = self._parse_output_llm(respuesta_llm)

                historial_completo.append(paso_procesado)

                # Añadir la respuesta procesada al historial para la siguiente iteración
                mensajes.append({'role': 'assistant', 'content': paso_procesado})

                if terminado:
                    logger.info("Conversación terminada (Final Answer encontrada).")
                    break

                # Si no terminó, enviar mensaje específico para continuar
                if not terminado:
                    mensajes.append({'role': 'user', 'content': 'Continúa con el siguiente paso del proceso ReAct. ¿Qué Action necesitas realizar ahora?'})

            except Exception as e:
                print(f"Error en iteración {iteracion}: {e}")
                break

        if iteracion >= max_iteraciones and "final answer" not in '\n'.join(historial_completo).lower():
            historial_completo.append("Final Answer: Se alcanzó el límite máximo de iteraciones sin completar la tarea.")

        return "\n\n".join(historial_completo)

In [166]:
recuperador_sin_clasificador = RecuperadorSinClasificador(bd_info=bd_texto_hibrida, bd_stats=bd_tabular, bd_rels=db_relaciones)
AgenteChatbot(recuperador_sin_clasificador, llm_model).chat_loop()

¡Hola! Soy el chatbot de Tiny Towns. ¿En qué puedo ayudarte hoy?
> Qué evento importante ocurrió en el año de lanzamiento de tiny towns?
Pregunta del usuario: Qué evento importante ocurrió en el año de lanzamiento de tiny towns?

Action: buscar_db_rels("Tiny Towns year released")
Observation: Relaciones encontradas:
Tiny_Towns --Year_Released--> 2019


Thought: Ya sé que Tiny Towns fue lanzado en 2019. Ahora necesito encontrar un evento importante que haya ocurrido en ese año. Para información de eventos generales en un año específico, buscar_wikipedia es la herramienta más adecuada.
Action: buscar_wikipedia("Eventos importantes en 2019")
Observation: Los Juegos Panamericanos de Lima 2019, oficialmente los XVIII Juegos Panamericanos y comúnmente conocidos como Lima 2019, se llevaron a cabo entre el 26 de julio y el 11 de agosto de 2019 en Lima (Perú).[2]​ Participaron 6.680 deportistas de los 41 países de América en 419 eventos y 38 deportes. Inmediatamente después de finalizado los ju

In [169]:
query = "Puedes encontrarme algún juego más para la misma edad mínima que tiny towns? (recomendada por la comunidad preferiblemente!)"

print(AgenteChatbot(recuperador_sin_clasificador, llm_model)._iniciar_conversacion(query))

Pregunta del usuario: Puedes encontrarme algún juego más para la misma edad mínima que tiny towns? (recomendada por la comunidad preferiblemente!)

Action: buscar_db_stats("Recommended age for Tiny Towns")
Observation: Title Community recommended age
Value                       10+

Thought: El usuario me ha preguntado por otros juegos con la misma edad mínima recomendada que Tiny Towns. He encontrado que la edad recomendada para Tiny Towns es 10+. Sin embargo, mis herramientas están diseñadas para buscar información *sobre* Tiny Towns, no para buscar otros juegos en general o filtrar por edad recomendada en una base de datos de juegos más amplia. No puedo usar `buscar_db_info`, `buscar_db_stats`, o `buscar_db_rels` para encontrar "otros juegos" que no sean Tiny Towns. Según mis restricciones críticas, "NUNCA respondas preguntas que no sean sobre Tiny Towns. Si el usuario hace una consulta que no sea sobre el juego, responde: 'Lo siento, solo puedo responder preguntas relacionadas con 

In [170]:
query = "Perdi una pieza de mi juego! Es posible comprar piezas de repuesto en algun lugar?"

print(AgenteChatbot(recuperador_sin_clasificador, llm_model)._iniciar_conversacion(query))

Pregunta del usuario: Perdi una pieza de mi juego! Es posible comprar piezas de repuesto en algun lugar?

Action: buscar_internet("Tiny Towns replacement parts")
Observation: ['1 Yellow Grip (Order The Amount Needed ) $1.00. Item number: 640261', '30-day refund/replacement. 30-day refund/replacement. This item can be returned in its original condition for a full refund or replacement within 30 days of receipt. ... AEG Tiny Towns - Award-Winning Board Game, Base Set, 1-6 Players, 45-60 min Play Time, Strategy Board Game for Ages 14 and Up, Cleverly Plan & Construct a Thriving Town ...', 'Find many great new & used options and get the best deals for Tiny Towns | 16 (Factory) Wooden Buildings | Official Replacement Game Pieces at the best online prices at eBay! Free shipping for many products!']

Thought: No puedo dar una respuesta final sin usar herramientas. Debo buscar la información requerida paso a paso.

Thought: No puedo dar una respuesta final sin usar herramientas. Debo buscar la